# Домашнее задание к занятию 
# "Классификация: Логистическая регрессия и SVM"

Имеются данные adult.csv (см. в материалах к занятию).
Целевой переменной является уровень дохода income (крайний правый столбец).
Описание признаков можно найти по ссылке www.cs.toronto.edu...etail.html
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

In [1]:
import numpy as np
import pandas as pd # Для работы с данными
import scipy.stats # При работе со статистикой
import matplotlib.pyplot as plt  # Библиотека для визуализации результатов

In [2]:
# чтение данных
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,class
0,2,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States,<=50K
1,3,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States,<=50K
2,2,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States,<=50K
3,3,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States,<=50K
4,1,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba,<=50K


### Изучение и подготовка данных

In [3]:
# информация о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      46033 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capitalgain     48842 non-null  int64 
 11  capitalloss     48842 non-null  int64 
 12  hoursperweek    48842 non-null  int64 
 13  native-country  47985 non-null  object
 14  class           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
# оценка количества пропущенных данных
for col in data.columns:
    missing = data[col].isna().mean()
    print(f'{col} - {missing: .1%}')

age -  0.0%
workclass -  5.7%
fnlwgt -  0.0%
education -  0.0%
education-num -  0.0%
marital-status -  0.0%
occupation -  5.8%
relationship -  0.0%
race -  0.0%
sex -  0.0%
capitalgain -  0.0%
capitalloss -  0.0%
hoursperweek -  0.0%
native-country -  1.8%
class -  0.0%


Присутствуют пропуски в следующих показателях:
workclass -  5.7%
occupation -  5.8%
native-country -  1.8%

Так как пропусков немного, то можно удалить эти строки.

In [5]:
# удаление строк с пропусками
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             45222 non-null  int64 
 1   workclass       45222 non-null  object
 2   fnlwgt          45222 non-null  int64 
 3   education       45222 non-null  object
 4   education-num   45222 non-null  int64 
 5   marital-status  45222 non-null  object
 6   occupation      45222 non-null  object
 7   relationship    45222 non-null  object
 8   race            45222 non-null  object
 9   sex             45222 non-null  object
 10  capitalgain     45222 non-null  int64 
 11  capitalloss     45222 non-null  int64 
 12  hoursperweek    45222 non-null  int64 
 13  native-country  45222 non-null  object
 14  class           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [6]:
# количество групп в каждой переменной
data.nunique()

age                   5
workclass             7
fnlwgt            26741
education            16
education-num        16
marital-status        7
occupation           14
relationship          6
race                  5
sex                   2
capitalgain           5
capitalloss           5
hoursperweek          5
native-country       41
class                 2
dtype: int64

In [7]:
# статистика по некатегориальным переменным
continuous =  data._get_numeric_data()
continuous.describe()

,age,fnlwgt,education-num,capitalgain,capitalloss,hoursperweek
count,45222.000000,4.522200e+04,45222.000000,45222.000000,45222.000000,45222.000000
mean,1.769537,1.897347e+05,10.118460,0.205055,0.116558,1.988059
std,1.266785,1.056392e+05,2.552881,0.756177,0.560539,0.875218
min,0.000000,1.349200e+04,1.000000,0.000000,0.000000,0.000000
25%,1.000000,1.173882e+05,9.000000,0.000000,0.000000,2.000000
50%,2.000000,1.783160e+05,10.000000,0.000000,0.000000,2.000000
75%,3.000000,2.379260e+05,13.000000,0.000000,0.000000,2.000000
max,4.000000,1.490400e+06,16.000000,4.000000,4.000000,4.000000


In [8]:
data['native-country'].value_counts()

# Holand-Netherlands соответствует всего одна строка, можно было бы пренебречь для уменьшения ошибки.

United-States                 41292
Mexico                          903
Philippines                     283
Germany                         193
Puerto-Rico                     175
Canada                          163
El-Salvador                     147
India                           147
Cuba                            133
England                         119
China                           113
Jamaica                         103
South                           101
Italy                           100
Dominican-Republic               97
Japan                            89
Guatemala                        86
Vietnam                          83
Columbia                         82
Poland                           81
Haiti                            69
Portugal                         62
Iran                             56
Taiwan                           55
Greece                           49
Nicaragua                        48
Peru                             45
Ecuador                     

### Модель логистической регрессии

In [9]:
from sklearn.linear_model import LogisticRegression

#### 1. Предобработка данных. Для работы с переменными в рамках задачи Логистической регрессии нужно преобразовать категориальные переменные в числа.

К категориальным данным относятся следующие признаки:
workclass, education, marital-status, occupation, relationship, race, sex, native-country, 
class - целевая переменная.

In [10]:
# список Categorical признаков (без целевой переменной)
categorical = data.iloc[:,:-1].select_dtypes('object').columns
print(categorical)

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')


In [11]:
# преобразование категориальных признаков в числа pd.get_dummies
# преобразование
X = pd.get_dummies (data, columns = categorical)

# столбец 'class' является целевой переменной, его необходимо удалить из набора признаков X

del X['class']
X.head()

,age,fnlwgt,education-num,capitalgain,capitalloss,hoursperweek,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,2,77516,13,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,83311,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,215646,9,0,0,2,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,3,234721,7,0,0,2,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,1,338409,13,0,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# целевая переменная (class) является категориальной 
# переведем значения столбца в числа, оставив один столбец
# LabelEncoder ставит в соответствие значению class (<=50K, >50K) 0 и 1

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [13]:
le.fit( data['class'])

LabelEncoder()

In [14]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [15]:
# le.transform( ['<=50K', '<=50K', '>50K']) # пример расшифровки
# le.inverse_transform([1, 0, 1]) # пример расшифровки

In [16]:
# записываем в целевую переменную y преобразованный столбец class

y = pd.Series (data = le.transform(data['class']))
y.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    1
8    1
9    1
dtype: int32

In [17]:
from sklearn.pipeline import make_pipeline # используем пайплайны для удобства
from sklearn.preprocessing import StandardScaler # стандартизация данных
from sklearn.model_selection import train_test_split # разделение данных на train/test

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)) # pipeline (со стандартизацией данных)

In [18]:
# model = LogisticRegression() # модель логистической регресии из scikit-learn (без стандартизации данных)

#### 2. Обучение. Поделим данные на обучающую выборку и тестовую.

In [19]:
# разделение на train/test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
# обучение модели на train

model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [21]:
# прогноз вероятности

model.predict_proba(X_test)

array([[0.23865805, 0.76134195],
       [0.93770459, 0.06229541],
       [0.87925192, 0.12074808],
       ...,
       [0.70250629, 0.29749371],
       [0.65200389, 0.34799611],
       [0.67811998, 0.32188002]])

#### 3. Оценка точности модели на тренировочной и тестовой выборках

In [22]:
# Оценка точности модели на тренировочной выборке

model.score(X_train, y_train) 

0.847417469594061

In [23]:
# Оценка точности модели на тестовой выборке

model.score(X_test,y_test)

0.8505933515147048

#### Без StandardScaler

In [24]:
model = LogisticRegression() # модель логистической регресии из scikit-learn (без стандартизации данных)

In [25]:
# обучение модели на train

model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [26]:
# Оценка точности модели на тренировочной выборке

model.score(X_train, y_train)

0.7525193492339283

In [27]:
# Оценка точности модели на тестовой выборке

model.score(X_test, y_test)

0.7513083216628584

### Модель SVM

In [28]:
from sklearn.svm import SVC

In [29]:
# используем пайплайны

clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) 
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [30]:
# прогноз

predictions = clf.predict(X_test)

In [31]:
# Оценка точности модели на тренировочной выборке

clf.score(X_train, y_train) 

0.8579371347338494

In [32]:
# Оценка точности модели на тестовой выборке

clf.score(X_test, y_test)

0.8488243532099948

#### Без StandardScaler 

In [33]:
svc = SVC(gamma='auto')
svc.fit(X_train, y_train)

SVC(gamma='auto')

In [34]:
predictions = svc.predict(X_test)

In [35]:
# Оценка точности модели на тренировочной выборке

svc.score(X_train, y_train) 

0.8643184331069341

In [36]:
# Оценка точности модели на тестовой выборке

svc.score(X_test, y_test)

0.7471069506891722